In [1]:
import xarray as xr
import numpy as np
from scipy.optimize import minimize

In [2]:
def optim_func(x, a, b, c):
    # An arbitrary function of x to optimize
    return (a / x ** 2) + b * np.abs(x) + c * np.sqrt(x)

@np.vectorize
def _optim_wrapper(a, b, c):
    min_result = minimize(
        optim_func, [1], (a, b, c),
        bounds=[(1e-2, None)]
    )
    
    return min_result.x, min_result.nit

# test
_optim_wrapper(0.56, 0.98, 0.12)

(array([1.02525762]), array(2))

In [3]:
# Construct an array
size = 50
a, b, c = np.random.rand(3, size, size)

da = xr.Dataset(
    data_vars = dict(
        a=(["x", "y"], a),
        b=(["x", "y"], b),
        c=(["x", "y"], c)
    ),
    coords=dict(
        x=np.linspace(0, 5),
        y=np.linspace(0, 5)
    )
)
da

<xarray.Dataset>
Dimensions:  (x: 50, y: 50)
Coordinates:
  * x        (x) float64 0.0 0.102 0.2041 0.3061 ... 4.694 4.796 4.898 5.0
  * y        (y) float64 0.0 0.102 0.2041 0.3061 ... 4.694 4.796 4.898 5.0
Data variables:
    a        (x, y) float64 0.5288 0.8793 0.9799 0.7285 ... 0.276 0.01222 0.7672
    b        (x, y) float64 0.06378 0.1858 0.109 0.2559 ... 0.6243 0.5108 0.8088
    c        (x, y) float64 0.2546 0.7316 0.5161 0.1799 ... 0.1774 0.8081 0.6628

This works with one output, but idk how to make it work with multiple outputs

In [4]:
# Apply optimization as a ufunc. xr.apply_ufunc returns a tuple so assign to variables
# with unpacking.
da["soln"], da["nit"] = xr.apply_ufunc(
    _optim_wrapper, da["a"], da["b"], da["c"],
    input_core_dims = [["x", "y"], ["x", "y"], ["x", "y"]],
    output_core_dims = [["x", "y"], ["x", "y"]]
)
da

<xarray.Dataset>
Dimensions:  (x: 50, y: 50)
Coordinates:
  * x        (x) float64 0.0 0.102 0.2041 0.3061 ... 4.694 4.796 4.898 5.0
  * y        (y) float64 0.0 0.102 0.2041 0.3061 ... 4.694 4.796 4.898 5.0
Data variables:
    a        (x, y) float64 0.5288 0.8793 0.9799 0.7285 ... 0.276 0.01222 0.7672
    b        (x, y) float64 0.06378 0.1858 0.109 0.2559 ... 0.6243 0.5108 0.8088
    c        (x, y) float64 0.2546 0.7316 0.5161 0.1799 ... 0.1774 0.8081 0.6628
    soln     (x, y) float64 1.891 1.541 1.875 1.647 ... 0.9166 0.2663 1.11
    nit      (x, y) int64 4 6 6 7 7 3 7 4 5 4 4 7 5 ... 5 5 6 5 7 7 4 7 5 3 6 4